### Directions
#### Complete Parts 1-3 below. 

We’ve estimated that this problem will take approx. five hours to complete. If you don't have time to finish that’s ok, just­ send what you have and explain what you planned on doing. We’re flexible on how you provide your work (e.g. link to GitHub repo, git bundle, zipped folder). Preference is for code to be written in Python. 
##### Data
The data for this problem can be downloaded from here (JSON recommended). Note that CVE-Modified and CVE-Recent feeds are not required.




In [18]:
import matplotlib

from run import Run
import pandas as pd

In [19]:
#mainObj = Run(collection='t2').fill_mongo()


### Part 1
Explore the data to get a better understanding of the content

### Guidance for Part 1
- Create a Data frame with one row per CVE id
- How many CVEs have CVSSv3 metrics versus only CVSSv2 metrics?
- Both CVSSv2 and CVSSv3 have the same set of impact metrics, i.e. Confidentiality, Integrity and Availability, however their values are slightly different. For example, CVSSv2 uses complete (C) to represent the highest level of impact, but CVSSv3 uses high (H) instead. Is it possible to directly map from CVSSv2 impact metric values to CVSSv3?


In [20]:
#
#
dfs = Run().fill_df()
dfV2,dfV3 = dfs.dfV2,dfs.dfV3
df = pd.merge( dfV3, dfV2, 'outer', '_id',suffixes=['_V3', '_V2'])


In [21]:
Δ_list = [['attackVector','AV_3'], ['accessVector', 'AV_2']]
dum_add = lambda ele: pd.get_dummies(df[ele[0]],prefix=ele[1])

frames = [dum_add(i) for i in Δ_list]

df = df.drop([i[0] for i in Δ_list], axis=1)

frames.append(df)

df = pd.concat(frames,axis=1)

In [22]:
dfV2_keys,dfV3_keys = dfs.dfV2.keys(),dfs.dfV3.keys()


transform_idx = df.T.index

for i in transform_idx:
    #print(i[:4])
    if i[:4] == 'AV_3':
        dfV3_keys = dfV3_keys.append(pd.Index([i]))
    if i[:4] == 'AV_2':
        dfV2_keys = dfV2_keys.append(pd.Index([i]))

dfV3_keys = dfV3_keys.drop('attackVector')
dfV2_keys = dfV2_keys.drop('accessVector')

setDf = dfV3_keys.union(dfV2_keys)#set(dfV3_keys)
maskPairs = []

for i in setDf: 
    
    substr_i = transform_idx[transform_idx.str.startswith(i+'_')]
    if len(substr_i) >0:
        maskPairs.append(substr_i)
    

#maskPairs

In [23]:
dfV2_keys

Index(['_id', 'version', 'vectorString', 'accessComplexity', 'authentication',
       'confidentialityImpact', 'integrityImpact', 'availabilityImpact',
       'baseScore', 'severity', 'exploitabilityScore', 'impactScore',
       'obtainAllPrivilege', 'obtainUserPrivilege', 'obtainOtherPrivilege',
       'userInteractionRequired', 'acInsufInfo', 'AV_2_ADJACENT_NETWORK',
       'AV_2_LOCAL', 'AV_2_NETWORK'],
      dtype='object')

In [200]:
list_ = []
for i in df.columns:
    if len(df[i].unique()) <10:
        print(df[i].unique(), i)
        '''for j in df[i].unique():
            #print(j,type(j))
            if type(j) != bool and type(j) != float and not j[0].isnumeric():# and not j.is:
                list_.append(j)
list_ = set(list_)
list_'''

<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8 AV_3_ADJACENT_NETWORK
<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8 AV_3_LOCAL
<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8 AV_3_NETWORK
<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8 AV_3_PHYSICAL
<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8 AV_2_ADJACENT_NETWORK
<IntegerArray>
[0, 1]
Length: 2, dtype: UInt8 AV_2_LOCAL
<IntegerArray>
[1, 0]
Length: 2, dtype: UInt8 AV_2_NETWORK
<FloatingArray>
[<NA>, 3.0, 3.1]
Length: 3, dtype: Float64 version_V3
<IntegerArray>
[<NA>, 1, 3]
Length: 3, dtype: Int64 attackComplexity
<IntegerArray>
[<NA>, 0, 1, 3]
Length: 4, dtype: Int64 privilegesRequired
<IntegerArray>
[<NA>, 1, 0]
Length: 3, dtype: Int64 userInteraction
<IntegerArray>
[<NA>, 0, 1]
Length: 3, dtype: Int64 scope
<IntegerArray>
[<NA>, 0, 3, 1]
Length: 4, dtype: Int64 confidentialityImpact_V3
<IntegerArray>
[<NA>, 0, 3, 1]
Length: 4, dtype: Int64 integrityImpact_V3
<IntegerArray>
[<NA>, 3, 0, 1]
Length: 4, dtype: Int64 availabilityImpact_V3
<I

In [25]:
cat_dict = {#'PHYSICAL':0 ,'LOCAL':1, 'NETWORK':2,'ADJACENT_NETWORK':3,
'NONE': 0, 'LOW':1, 'MEDIUM':2,'HIGH':3, 'CRITICAL':4,
'PARTIAL': 1, 'COMPLETE':2,
'SINGLE' :1, 'MULTIPLE' :2,
'UNCHANGED':0 ,'CHANGED':1,
'REQUIRED':1,
False:0, True:1
}

In [26]:
cat_dict.values()

dict_values([0, 1, 2, 3, 4, 1, 2, 1, 2, 0, 1, 1, 0, 1])

In [27]:
df.replace(cat_dict, inplace=True)

In [28]:
f = df.filter(regex='version*').columns
df[f] = df[f].astype(float)

In [29]:
df = df.convert_dtypes()

In [30]:
#df.plot(kind = 'scatter', x = 'confidentialityImpact_x', y = 'confidentialityImpact_y')

In [31]:
df.to_csv('out.csv')

### Part 2
Predict the CVSSv3 Scope metric for CVEs without a CVSSv3 vector.

In [32]:

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis  import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay,confusion_matrix, classification_report, mean_absolute_error
from sklearn.preprocessing import StandardScaler

from pprint import pprint


In [33]:

cssv3 = []
cssv2 = []
dfV3_keys_cp = dfV3_keys.drop(['_id','vectorString'])
dfV2_keys_cp = dfV2_keys.drop(['_id','vectorString'])
for i in maskPairs:
    if i[0][:-3] in dfV3_keys and i[0][:-3] != 'vectorString':
        cssv3.append(i[0])
        dfV3_keys_cp = dfV3_keys_cp.drop([i[0][:-3]])

    
    if i[0][:-3] in dfV2_keys and i[0][:-3] != 'vectorString':
        cssv2.append(i[1])
        dfV2_keys_cp = dfV2_keys_cp.drop([i[0][:-3]])

cssv3 = dfV3_keys_cp.union(cssv3)  
cssv2 = dfV2_keys_cp.union(cssv2)   


In [153]:
dfNoNan = df.loc[:,df.dtypes != 'object'].drop('acInsufInfo',axis=1).dropna()

In [154]:
X = dfNoNan[cssv2.drop('acInsufInfo')]
y = dfNoNan['scope'].astype(int)


In [155]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1234) 

In [156]:
ncol = len(X_train.columns)

In [157]:


model_RFC = RandomForestClassifier()
model_SVC = svm.SVC()
model_MLP = MLPClassifier()
model_LR =  LogisticRegression()
model_LDA =  LinearDiscriminantAnalysis()
model_KNN =  KNeighborsClassifier()
model_CART =  DecisionTreeClassifier()
model_NB =  GaussianNB()

#sc = StandardScaler()

#X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)


mod_list = [
    
    #model_SVC,
    model_MLP,
    model_LR,
    model_LDA,
    model_KNN,
    model_CART,
    model_RFC,
    model_NB]



In [158]:
df.scope.value_counts()

0    83029
1    16864
Name: scope, dtype: Int64

In [185]:
def model_info(model):
    print('~~~ fitting model')
    f = model.fit(X_train.values, y_train.values)
    print('~~~ predicting values')
    pred = model.predict(X_test.values)
    print('~~~ checking validity')
    sc = f.score(X_test.values, y_test.values)
    m = mean_absolute_error(y_test.values, pred)
    c = confusion_matrix(y_test.values, pred)
    dict_ = {'model': f, 'score' : sc, 'prediction': pred, 'MAE' : m, 'Confusion Matrix': c}
    return dict_
    

In [201]:
def model_dict():

    mod_dict ={}
    idx = 0
    for i in mod_list: 
        mod_type = i.__str__()
        
        print(f"\n~~~~~~~~~~~~~~~~~~~~~~~~~\nWorking on {mod_type}")
        
        mod_info = model_info(i)
        mod_dict[mod_type] =  (idx ,mod_info)
        idx+=1
        pprint(mod_info)
        
    return mod_dict

mod_dict_p2 = model_dict()



~~~~~~~~~~~~~~~~~~~~~~~~~
Working on SVC()
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[19920,   524],
       [  960,  3348]]),
 'MAE': 0.05995475113122172,
 'model': SVC(),
 'prediction': array([0, 0, 0, ..., 0, 1, 0]),
 'score': 0.9400452488687783}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on MLPClassifier()
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[19945,   499],
       [  927,  3381]]),
 'MAE': 0.0576115061409179,
 'model': MLPClassifier(),
 'prediction': array([0, 0, 0, ..., 0, 1, 0]),
 'score': 0.9423884938590821}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on LogisticRegression()
~~~ fitting model


/home/potz/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[19820,   624],
       [ 1187,  3121]]),
 'MAE': 0.07316580478345185,
 'model': LogisticRegression(),
 'prediction': array([0, 0, 0, ..., 0, 1, 0]),
 'score': 0.9268341952165482}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on LinearDiscriminantAnalysis()
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[19496,   948],
       [  902,  3406]]),
 'MAE': 0.07474143503555268,
 'model': LinearDiscriminantAnalysis(),
 'prediction': array([0, 0, 0, ..., 0, 1, 0]),
 'score': 0.9252585649644474}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on KNeighborsClassifier()
~~~ fitting model
~~~ predicting values
~~~ checking validity
{'Confusion Matrix': array([[19864,   580],
       [  894,  3414]]),
 'MAE': 0.05955074337427278,
 'model': KNeighborsClassifier(),
 'prediction': array([0, 0, 0, ..., 0, 1, 0]),
 'score': 0.9404492566257272}

~~~~~~~~~~~~~~~~~~~~~~~~~
Working on DecisionTreeClassifier()
~~~ fittin

In [207]:
mod_dict_p2['SVC()']

(0,
 {'model': SVC(),
  'score': 0.9400452488687783,
  'prediction': array([0, 0, 0, ..., 0, 1, 0]),
  'MAE': 0.05995475113122172,
  'Confusion Matrix': array([[19920,   524],
         [  960,  3348]])})

In [208]:
ytest_vals = {}
for i in mod_list:
    #print(i)
    pred = pd.DataFrame(mod_dict_p2[str(i)][1]['prediction'])#pd.DataFrame(i.predict(Xnew))
    ytest_vals[i.__str__()] = pred.value_counts(), pred

In [209]:
for i in ytest_vals:
    print(ytest_vals[i][0][1]/ytest_vals[i][0][0])

0.1854406130268199
0.1858949789191261
0.17827390869710097
0.213452299245024
0.19240774641102226
0.1871462829736211
0.1876589415095245
1.59808963997061


In [210]:
best_mod= ('',0)

for i in mod_dict_p2:
    #print(mod_dict_p2[i])
    if mod_dict_p2[i][1]['score'] > best_mod[1]:
        best_mod = mod_dict_p2[i][0], mod_dict_p2[i][1]['score']

print(best_mod)
work_model = mod_list[best_mod[0]]

(6, 0.9429137039431157)


In [211]:
# prediction df with scope = nan
'''df_pred = df[cssv2.append(pd.Index(['scope'])).drop('acInsufInfo')]
df_pred[]
print(df_pred)'''
df_pred = df[df['scope'].isna()]

Xnew = df_pred[cssv2.drop('acInsufInfo')].dropna()
#Xnew = sc.transform(Xnew)
ynew = work_model.predict(Xnew)
#Xnew

/home/potz/.local/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [212]:
ynew_vals = {}
for i in mod_list:
    pred = pd.DataFrame(i.predict(Xnew.values))
    ynew_vals[i.__str__()] = pred.value_counts(), pred

In [192]:
for i in ynew_vals:
    print(ynew_vals[i][0][1]/ynew_vals[i][0][0])

0.16254273156438223
0.16930004093327874
0.16748406081412456
0.184721300597213
0.17217890849405007
0.17131376086599967
0.1727563839395681
1.0945886493620764


In [139]:
pd.DataFrame(ynew).value_counts()

0    14273
1       10
dtype: int64


### Part 3
Predict the CVSSv3 Confidentiality, Integrity and Availability metrics for CVEs without a CVSSv3 vector.

### Guidance for Parts 2-3
- What type of learning problem is this? What is the target? 
- How would you build the training / validation / testing dataset?
- Which evaluation metrics would you use?
- What simple model might be used for this problem? Could this be improved upon with a more complex solution?


In [194]:
cssv3

Index(['AV_3_ADJACENT_NETWORK', 'AV_3_LOCAL', 'AV_3_NETWORK', 'AV_3_PHYSICAL',
       'attackComplexity', 'availabilityImpact_V3', 'baseScore_V3',
       'baseSeverity', 'confidentialityImpact_V3', 'exploitabilityScore_V3',
       'impactScore_V3', 'integrityImpact_V3', 'privilegesRequired', 'scope',
       'userInteraction', 'version_V3'],
      dtype='object')

In [204]:
response = pd.Index(['confidentialityImpact_V3', 'integrityImpact_V3', 'availabilityImpact_V3'])
X = dfNoNan[cssv2.drop('acInsufInfo')]
y = dfNoNan[response]#.astype(int)

In [205]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1234) 

In [206]:
mod_dict_p3 = model_dict()


~~~~~~~~~~~~~~~~~~~~~~~~~
Working on SVC()
~~~ fitting model


ValueError: y should be a 1d array, got an array of shape (74254, 3) instead.